In [ ]:
import numpy as np
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory



In [1]:
a = "kata muhammad dia allah yang maha esa".split()
b = "allah tidak anak dan tidak pula anak".split()
c = "dari jahat bisik setan yang sembunyi".split()
d = "yang bisik jahat ke dalam dada manusia".split()
e = "dari golong jin dan manusia".split()
query = "apakah sifat Allah?"

In [2]:
docs = [a,b,c,d,e]

In [ ]:


def tfidf_normal (word, sentence):
    freq = sentence.count(word)
    tf =  freq / len(sentence)
    idf = np.log10(len(docs)/sum([1 for doc in docs if word in doc]))

    return round(tf*idf, 4)

In [4]:
N = len(docs)
avgdl = sum(len(sentence) for sentence in docs) / N
print(N)
print(avgdl)

5
6.4


In [5]:
def tf_idf(word, sentence, k=1.5, b=0.75):
    n_t = sum([1 for doc in docs if word in doc])
    idf = (((N-n_t) + 0.5) / (n_t + 0.5))
    idf = np.log10(idf)
    # print("N    :", N)
    # print("n_t  :", n_t)
    # print("idf  :",idf)
    # print("\n")

    f_td = sentence.count(word)
    tf = (f_td*(k+1)) / (f_td + (k * (1 - b + (b * ((len(sentence) / avgdl))))))
    # print("f_td :",f_td)
    # print("d    :",len(sentence) )
    # print("atas :",(f_td*(k+1)) )
    # print("bawah:",(f_td + (k * (1 - b + (b * ((len(sentence) / avgdl)))))))
    # print("tf   :",tf)

    

    return round(tf*idf,6)

In [6]:
for doc in docs :
    print(tf_idf("siapa", doc))

0.0
0.0
0.0
0.0
0.0


In [7]:
for doc in docs :
    print(tf_idf("allah", doc))

0.140213
0.140213
0.0
0.0
0.0


In [8]:
for doc in docs :
    print(tf_idf("tuhan", doc))

0.0
0.0
0.0
0.0
0.0


In [9]:
def bm25(query, sentence):
    result = sum(tf_idf(word, sentence) for word in query)
    
    return result

In [10]:
kueri = ["siapa", "muhammad", "allah"]
for doc in docs:
    print(bm25(kueri, doc))

0.598021
0.140213
0.0
0.0
0.0


# Dataset Manualisasi

In [ ]:

df = pd.read_csv("data_manualisasi.csv") 
df

,id,suraId,verseID,ayahText,indoText,readText
0,6221,112,1,قُلْ هُوَ اللّٰهُ اَحَدٌۚ,"Katakanlah (Muhammad), “Dialah Allah, Yang Mah...",qul huwallāhu aḥad
1,6223,112,3,لَمْ يَلِدْ وَلَمْ يُوْلَدْۙ,(Allah) tidak beranak dan tidak pula diperanak...,lam yalid wa lam yụlad
2,6233,114,4,مِنْ شَرِّ الْوَسْوَاسِ ەۙ الْخَنَّاسِۖ,"dari kejahatan (bisikan) setan yang bersembunyi,",min syarril-waswāsil-khannās
3,6234,114,5,الَّذِيْ يُوَسْوِسُ فِيْ صُدُوْرِ النَّاسِۙ,yang membisikkan (kejahatan) ke dalam dada man...,allażī yuwaswisu fī ṣudụrin-nās
4,6235,114,6,مِنَ الْجِنَّةِ وَالنَّاسِ,dari (golongan) jin dan manusia.”,minal-jinnati wan-nās


## Preproses

In [12]:
kueri = "Siapa Allah? Tuhan?"

In [ ]:
# Preproses tanpa stemming



def preprocess_text_no_stem(text):
    if pd.isna(text):  # Cek jika NaN
        return ""
    

    text = text.lower()  # lowercasing
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # hapus tanda baca, karakter spesial, dan angka
    tokens = text.split()  # Tokenisasi manual (split spasi)

    return tokens

# preprocessing indoText  ke kolom processed_text
df["processed_text_no_stem"] = df["indoText"].apply(preprocess_text_no_stem)


# Tampilkan hasil
print(df[["indoText", "processed_text_no_stem"]].head())


                                            indoText  \
0  Katakanlah (Muhammad), “Dialah Allah, Yang Mah...   
1  (Allah) tidak beranak dan tidak pula diperanak...   
2   dari kejahatan (bisikan) setan yang bersembunyi,   
3  yang membisikkan (kejahatan) ke dalam dada man...   
4                  dari (golongan) jin dan manusia.”   

                              processed_text_no_stem  
0  [katakanlah, muhammad, dialah, allah, yang, ma...  
1  [allah, tidak, beranak, dan, tidak, pula, dipe...  
2  [dari, kejahatan, bisikan, setan, yang, bersem...  
3  [yang, membisikkan, kejahatan, ke, dalam, dada...  
4                [dari, golongan, jin, dan, manusia]  


In [14]:
preprocess_text_no_stem("(Allah) tidak beranak dan tidak pula diperanakkan.")


['allah', 'tidak', 'beranak', 'dan', 'tidak', 'pula', 'diperanakkan']

In [ ]:
# Coba Stemming


def preprocess_text(text):
    if pd.isna(text):
        return []
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    
    text = text.lower()  # lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # hapus tanda baca, karakter spesial, dan angka
    tokens = text.split()  # tokenisasi split spasi
    
    # stemming untuk setiap token
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    
    return stemmed_tokens

In [16]:
preprocess_text("(Allah) tidak beranak dan tidak pula diperanakkan.")


['allah', 'tidak', 'anak', 'dan', 'tidak', 'pula', 'anak']

In [17]:
# preprocessing indoText  ke kolom processed_text
df["processed_text"] = df["indoText"].apply(preprocess_text)

# Tampilkan hasil
print(df[["indoText", "processed_text"]].head())

                                            indoText  \
0  Katakanlah (Muhammad), “Dialah Allah, Yang Mah...   
1  (Allah) tidak beranak dan tidak pula diperanak...   
2   dari kejahatan (bisikan) setan yang bersembunyi,   
3  yang membisikkan (kejahatan) ke dalam dada man...   
4                  dari (golongan) jin dan manusia.”   

                                   processed_text  
0   [kata, muhammad, dia, allah, yang, maha, esa]  
1    [allah, tidak, anak, dan, tidak, pula, anak]  
2     [dari, jahat, bisik, setan, yang, sembunyi]  
3  [yang, bisik, jahat, ke, dalam, dada, manusia]  
4               [dari, golong, jin, dan, manusia]  


In [18]:
df

,id,suraId,verseID,ayahText,indoText,readText,processed_text_no_stem,processed_text
0,6221,112,1,قُلْ هُوَ اللّٰهُ اَحَدٌۚ,"Katakanlah (Muhammad), “Dialah Allah, Yang Mah...",qul huwallāhu aḥad,"[katakanlah, muhammad, dialah, allah, yang, ma...","[kata, muhammad, dia, allah, yang, maha, esa]"
1,6223,112,3,لَمْ يَلِدْ وَلَمْ يُوْلَدْۙ,(Allah) tidak beranak dan tidak pula diperanak...,lam yalid wa lam yụlad,"[allah, tidak, beranak, dan, tidak, pula, dipe...","[allah, tidak, anak, dan, tidak, pula, anak]"
2,6233,114,4,مِنْ شَرِّ الْوَسْوَاسِ ەۙ الْخَنَّاسِۖ,"dari kejahatan (bisikan) setan yang bersembunyi,",min syarril-waswāsil-khannās,"[dari, kejahatan, bisikan, setan, yang, bersem...","[dari, jahat, bisik, setan, yang, sembunyi]"
3,6234,114,5,الَّذِيْ يُوَسْوِسُ فِيْ صُدُوْرِ النَّاسِۙ,yang membisikkan (kejahatan) ke dalam dada man...,allażī yuwaswisu fī ṣudụrin-nās,"[yang, membisikkan, kejahatan, ke, dalam, dada...","[yang, bisik, jahat, ke, dalam, dada, manusia]"
4,6235,114,6,مِنَ الْجِنَّةِ وَالنَّاسِ,dari (golongan) jin dan manusia.”,minal-jinnati wan-nās,"[dari, golongan, jin, dan, manusia]","[dari, golong, jin, dan, manusia]"


In [19]:
kueri = preprocess_text(kueri)


In [20]:
row = df[df["id"] == 6221]

processed_text = row["processed_text"].values[0]
bm25(kueri, processed_text)

0.140213

In [21]:
bm25(kueri, df["processed_text"][1])

0.140213

In [22]:
dict_scores = {
    row["id"]: bm25(kueri, row["processed_text"])
    for _, row in df.iterrows()
}
dict_scores

{6221: 0.140213, 6223: 0.140213, 6233: 0.0, 6234: 0.0, 6235: 0.0}

In [23]:
dict_top5 = dict(sorted(dict_scores.items(), key=lambda x:x[1], reverse=True)[:5])
dict_top5

{6221: 0.140213, 6223: 0.140213, 6233: 0.0, 6234: 0.0, 6235: 0.0}

In [24]:
# display top 5
top5_ayat = df[df['id'].isin(dict_top5.keys())] 
top5_ayat

,id,suraId,verseID,ayahText,indoText,readText,processed_text_no_stem,processed_text
0,6221,112,1,قُلْ هُوَ اللّٰهُ اَحَدٌۚ,"Katakanlah (Muhammad), “Dialah Allah, Yang Mah...",qul huwallāhu aḥad,"[katakanlah, muhammad, dialah, allah, yang, ma...","[kata, muhammad, dia, allah, yang, maha, esa]"
1,6223,112,3,لَمْ يَلِدْ وَلَمْ يُوْلَدْۙ,(Allah) tidak beranak dan tidak pula diperanak...,lam yalid wa lam yụlad,"[allah, tidak, beranak, dan, tidak, pula, dipe...","[allah, tidak, anak, dan, tidak, pula, anak]"
2,6233,114,4,مِنْ شَرِّ الْوَسْوَاسِ ەۙ الْخَنَّاسِۖ,"dari kejahatan (bisikan) setan yang bersembunyi,",min syarril-waswāsil-khannās,"[dari, kejahatan, bisikan, setan, yang, bersem...","[dari, jahat, bisik, setan, yang, sembunyi]"
3,6234,114,5,الَّذِيْ يُوَسْوِسُ فِيْ صُدُوْرِ النَّاسِۙ,yang membisikkan (kejahatan) ke dalam dada man...,allażī yuwaswisu fī ṣudụrin-nās,"[yang, membisikkan, kejahatan, ke, dalam, dada...","[yang, bisik, jahat, ke, dalam, dada, manusia]"
4,6235,114,6,مِنَ الْجِنَّةِ وَالنَّاسِ,dari (golongan) jin dan manusia.”,minal-jinnati wan-nās,"[dari, golongan, jin, dan, manusia]","[dari, golong, jin, dan, manusia]"


In [ ]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string

import numpy as np

def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc

In [ ]:
bm25 = BM25Okapi(tokenized_corpus)
bm25_scores = bm25.get_scores(bm25_tokenizer(query))
top_n = np.argpartition(bm25_scores, -10)[-10:]
bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)